In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
#!cp drive/MyDrive/MMDS-data/spark-3.1.1-bin-hadoop3.2.tgz .
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext

sc = SparkContext(master="local", appName="midterm")

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StringType, ArrayType, StructField

sqlc = SQLContext(sc)

# Câu 1: Đếm món hàng

In [ ]:
#đọc dữ liệu data.csv
data = sc.textFile('/content/drive/MyDrive/BigData/Midterm/data.csv')

In [ ]:
# Tách dòng tiêu đề
header = data.first()
lines = data.filter(lambda line: line != header)

In [ ]:
# Tạo RDD mới chứa thông tin về mã khách hàng, ngày mua, tháng mua, năm mua và món hàng
data_rdd = lines.map(lambda line: line.split(",")) \
               .map(lambda x: (x[0], x[1], int(x[3]), int(x[4]), int(x[5]), int(x[6])))

In [ ]:
from operator import add

def count_by_key(data_rdd, key_indices):
    counts = data_rdd.map(lambda x: (tuple([x[i] for i in key_indices]), 1)) \
                     .reduceByKey(add).map(lambda line: ','.join([str(x) for x in line[0]]) + ',' + str(line[1]))
    return counts
    
# Đếm số lượng món hàng mỗi người khách mua theo từng ngày
day_counts = count_by_key(data_rdd, [0, 1])

# Đếm số lượng món hàng mỗi người khách mua theo từng tháng
month_counts = count_by_key(data_rdd, [0, 3])

# Đếm số lượng món hàng mỗi người khách mua theo từng năm
year_counts = count_by_key(data_rdd, [0, 2])

In [ ]:
# Lưu kết quả vào các thư mục counters_day, counters_month và counters_year
day_counts.saveAsTextFile("counters_day")
month_counts.saveAsTextFile("counters_month")
year_counts.saveAsTextFile("counters_year")

In [ ]:
# Lọc dữ liệu để lấy mã khách hàng, ngày mua hàng và tên hàng
data_filtered = data.map(lambda line: line.split(",")) \
                    .map(lambda fields: (fields[0], fields[1], fields[2]))

# Tạo một tuple (ngày, số lượng) từ tên hàng
daily_count = data_filtered.map(lambda fields: ((fields[1], fields[2]), 1)) \
                           .reduceByKey(lambda x, y: x + y)

# Tìm số lượng món hàng lớn nhất trong từng ngày
max_daily_count = daily_count.map(lambda fields: (fields[0][0], (fields[0][1], fields[1]))) \
                             .groupByKey() \
                             .mapValues(lambda x: max(x, key=lambda y: y[1])) \
                             .map((lambda line: str(line[0]) +',' + str(line[1][0]) + ',' + str(line[1][1])))

In [ ]:
max_daily_count.saveAsTextFile("max_day")

In [ ]:
# Đọc kết quả từ các thư mục counters_day, counters_month, counters_year và max_day
counter_day_rdd = sc.textFile("counters_day")
counter_month_rdd = sc.textFile("counters_month")
counter_year_rdd = sc.textFile("counters_year")
max_day_rdd = sc.textFile("max_day")

In [ ]:
# Hàm để tạo DataFrame từ RDD
def create_dataframe_from_rdd(rdd, header):
    return sqlc.createDataFrame(rdd.map(lambda x: x.split(",")).map(lambda x: (x[0], x[1], int(x[2]))), header)

# Tạo DataFrame từ RDD
header_day = ["Member_number", "Date", "Count"]
df_day = create_dataframe_from_rdd(counter_day_rdd, header_day)

header_month = ["Member_number", "Month", "Count"]
df_month = create_dataframe_from_rdd(counter_month_rdd, header_month)

header_year = ["Member_number", "Year", "Count"]
df_year = create_dataframe_from_rdd(counter_year_rdd, header_year)

header_max_day = ["Date", "Item", "Count"]
df_max_day = create_dataframe_from_rdd(max_day_rdd, header_max_day)

In [ ]:
# Hiển thị kết quả
df_day.show()
df_month.show()
df_year.show()
df_max_day.show()

+-------------+----------+-----+
|Member_number|      Date|Count|
+-------------+----------+-----+
|         1249|01/01/2014|    2|
|         1381|01/01/2014|    2|
|         1440|01/01/2014|    2|
|         1659|01/01/2014|    2|
|         1789|01/01/2014|    2|
|         1922|01/01/2014|    2|
|         2226|01/01/2014|    2|
|         2237|01/01/2014|    2|
|         2351|01/01/2014|    2|
|         2542|01/01/2014|    2|
|         2610|01/01/2014|    3|
|         2709|01/01/2014|    2|
|         2727|01/01/2014|    2|
|         2943|01/01/2014|    2|
|         2974|01/01/2014|    3|
|         3681|01/01/2014|    3|
|         3797|01/01/2014|    2|
|         3942|01/01/2014|    3|
|         3956|01/01/2014|    4|
|         4260|01/01/2014|    2|
+-------------+----------+-----+
only showing top 20 rows

+-------------+-----+-----+
|Member_number|Month|Count|
+-------------+-----+-----+
|         1249|    1|    2|
|         1381|    1|    2|
|         1440|    1|    2|
|         1659

# Câu 2: Giỏ hàng

In [ ]:
data = sc.textFile("/content/drive/MyDrive/BigData/Midterm/data.csv")

In [ ]:
rdd_2 = data.map(lambda x : x.split(",")).map(lambda x : ((x[0], x[1]), x[2])).reduceByKey(lambda x, y: x + "," + y)
rdd_2 = rdd_2.map(lambda x : str(x[0][0]) + ";" + x[0][1] + ";" + str(x[1]))
rdd_2.saveAsTextFile("/content/baskets")

In [ ]:
import os
if os.path.exists('/content/baskets/part-00000'):
  sqlc.read.csv('/content/baskets/part-00000', header=True, sep=';').show()

+-------------+----------+--------------------+
|Member_number|      Date|     itemDescription|
+-------------+----------+--------------------+
|         1249|01/01/2014| citrus fruit,coffee|
|         1381|01/01/2014|           curd,soda|
|         1440|01/01/2014|other vegetables,...|
|         1659|01/01/2014|specialty chocola...|
|         1789|01/01/2014|hamburger meat,ca...|
|         1922|01/01/2014|tropical fruit,ot...|
|         2226|01/01/2014|sausage,bottled w...|
|         2237|01/01/2014|bottled water,Ins...|
|         2351|01/01/2014|cleaner,shopping ...|
|         2542|01/01/2014|sliced cheese,bot...|
|         2610|01/01/2014|hamburger meat,bo...|
|         2709|01/01/2014|yogurt,frozen veg...|
|         2727|01/01/2014|hamburger meat,fr...|
|         2943|01/01/2014|whole milk,flower...|
|         2974|01/01/2014|berries,whipped/s...|
|         3681|01/01/2014|onions,whipped/so...|
|         3797|01/01/2014|  waffles,whole milk|
|         3942|01/01/2014|other vegetabl

## Câu 2A

In [ ]:
from pyspark.sql.functions import split

In [ ]:
customer_id = input("Nhập mã số khách hàng: ")
purchase_date = input("Nhập ngày mau (dd/mm/yyyy): ")

df = rdd_2.map(lambda x: x.split(';')).toDF(['Customer', 'Purchase Date', 'Basket Items'])

basket = df.filter((df['Customer'] == customer_id) & (df['Purchase Date'] == purchase_date))

if basket.count() == 0:
    print("Không tìm thấy giỏ hàng.")
else:
    print("Giỏ hàng của khách hàng này:")
    items = basket.select('Basket Items').rdd.flatMap(lambda x: x).collect()[0].split(',')
    for item in items:
        print(item.strip())

Nhập mã số khách hàng: 2974
Nhập ngày mau (dd/mm/yyyy): 01/01/2014
Giỏ hàng của khách hàng này:
berries
whipped/sour cream
bottled water


## Câu 2B

In [ ]:
from datetime import datetime

In [ ]:
customer_id = input("Nhập mã số khách hàng: ")
n = int(input("Nhập số lượng giỏ hàng tối đa để hiển thị: "))

baskets = rdd_2.filter(lambda x: x.startswith(customer_id)).sortBy(lambda x: datetime.strptime(x.split(";")[1], "%d/%m/%Y"), ascending=False).take(n)

if not baskets:
    print("Không tìm thấy giỏ hàng.")
else:
    print("{:<15s}{:<34s}{:<70s}".format("Member_number", "Date", "itemDescription"))
    for basket in baskets:
        items = basket.split(";")[2].split(",")
        items_str = ", ".join(items)
        print("{:<15s}{:<34s}{:<70s}".format(customer_id, basket.split(";")[1], items_str))

Nhập mã số khách hàng: 2974
Nhập số lượng giỏ hàng tối đa để hiển thị: 3
Member_number  Date                              itemDescription                                                       
2974           24/05/2014                        tropical fruit, frozen vegetables                                     
2974           09/02/2014                        pastry, waffles                                                       
2974           01/01/2014                        berries, whipped/sour cream, bottled water                            


# Câu 3: Tập phổ biến

In [ ]:
dfBaskets = sqlc.read.csv("/content/baskets/part-00000", header=True, sep=";")

In [ ]:
dfBaskets = dfBaskets.rdd.map(lambda x : [x[0], x[1], x[2].split(",")]).map(lambda x : [x[0], x[1], list(dict.fromkeys(x[2]))]).toDF(["Member_number", "Date", "Items"])
dfBaskets.show()

+-------------+----------+--------------------+
|Member_number|      Date|               Items|
+-------------+----------+--------------------+
|         1249|01/01/2014|[citrus fruit, co...|
|         1381|01/01/2014|        [curd, soda]|
|         1440|01/01/2014|[other vegetables...|
|         1659|01/01/2014|[specialty chocol...|
|         1789|01/01/2014|[hamburger meat, ...|
|         1922|01/01/2014|[tropical fruit, ...|
|         2226|01/01/2014|[sausage, bottled...|
|         2237|01/01/2014|[bottled water, I...|
|         2351|01/01/2014|[cleaner, shoppin...|
|         2542|01/01/2014|[sliced cheese, b...|
|         2610|01/01/2014|[hamburger meat, ...|
|         2709|01/01/2014|[yogurt, frozen v...|
|         2727|01/01/2014|[hamburger meat, ...|
|         2943|01/01/2014|[whole milk, flow...|
|         2974|01/01/2014|[berries, whipped...|
|         3681|01/01/2014|[onions, whipped/...|
|         3797|01/01/2014|[waffles, whole m...|
|         3942|01/01/2014|[other vegetab

In [ ]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="Items",
                    minSupport=0.01, 
                    minConfidence=0.1)
model = fpGrowth.fit(dfBaskets)
model.freqItemsets.show()

+----------------+----+
|           items|freq|
+----------------+----+
|        [coffee]| 128|
|   [butter milk]|  68|
| [specialty bar]|  56|
|         [flour]|  47|
|      [UHT-milk]|  89|
|     [ice cream]|  51|
|  [frozen meals]|  66|
|        [pastry]| 224|
|     [margarine]| 152|
|          [pork]|  77|
|       [napkins]| 106|
|[tropical fruit]| 187|
|   [soft cheese]|  49|
|       [chicken]|  62|
|[hamburger meat]|  74|
|     [chocolate]| 102|
|  [citrus fruit]| 143|
|    [rolls/buns]| 436|
| [bottled water]| 275|
|  [bottled beer]| 178|
+----------------+----+
only showing top 20 rows



In [ ]:
model.associationRules.show()

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



## Câu 3A

In [ ]:
support_threshold = float(input("Nhập ngưỡng support: "))
fpGrowth = FPGrowth(minSupport=support_threshold, itemsCol="Items")
model = fpGrowth.fit(dfBaskets)
model.freqItemsets.withColumnRenamed("freq", "support").show()


Nhập ngưỡng support: 0.1
+------------------+-------+
|             items|support|
+------------------+-------+
|      [rolls/buns]|    436|
|[other vegetables]|    463|
|      [whole milk]|    543|
+------------------+-------+



## Câu 3B

In [ ]:
from pyspark.sql.functions import array_contains

In [ ]:
fpGrowth = FPGrowth(minSupport=0.1, itemsCol="Items")
model = fpGrowth.fit(dfBaskets)
rules = model.associationRules

In [ ]:
rules.show()

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



In [ ]:
def P(dfBaskets, y):
  b = dfBaskets.rdd.map(lambda row: row.asDict()).collect()
  freq_y = 0
  for i in range(len(b)):
    if(set(y).issubset(b[i]["Items"])):
      freq_y += 1
  return freq_y/len(b)

In [ ]:
from functools import reduce

x = input("Enter item x: ")
y = input("Enter item y: ")

x = x.split(',')
y = y.split(',')

# print(x)
# print(y)

# confidence_x_y: return [Row(confidence=0.12151067)], [Row(confidence=0.12151067)], ...
confidence_x_y = rules.filter(reduce(lambda x, y: x & y, [array_contains(rules.antecedent, c) for c in x]) & reduce(lambda x, y: x & y, [array_contains(rules.consequent, c) for c in y])) \
                    .select("confidence").collect()


if len(confidence_x_y) > 0:
    print("Confidence({} => {}) = {}".format(x, y, confidence_x_y[0][0]))

    # interest = conf(x, y) - P(y)
    interest_x_y = confidence_x_y[0][0] - P(dfBaskets, x, y)
    print("Interest({} => {}) = {}".format(x, y, interest_x_y))
else:
    print("No association rule found for items {} and {}.".format(x, y))

Enter item x: ice-cream
Enter item y: yogurt
No association rule found for items ['ice-cream'] and ['yogurt'].


# Câu 4: Giỏ hàng thành Vector

In [ ]:
dfMembers = sqlc.read.csv("/content/drive/MyDrive/BigData/Midterm/data.csv", header=True)
dfMembers = dfMembers.select(["Member_number", "itemDescription"]).withColumnRenamed("itemDescription","Items")
dfMembers = dfMembers.rdd.groupByKey().mapValues(lambda x : ','.join([str(it) for it in set(x)])).toDF(["Member_number","Items"])

In [ ]:
dfMembers.show()

+-------------+--------------------+
|Member_number|               Items|
+-------------+--------------------+
|         1249| coffee,citrus fruit|
|         1381|coffee,other vege...|
|         1440|yogurt,other vege...|
|         1659|yogurt,specialty ...|
|         1789|hamburger meat,ca...|
|         1922|yogurt,other vege...|
|         2226|sausage,bottled w...|
|         2237|sliced cheese,flo...|
|         2351|shopping bags,cle...|
|         2542|bottled water,sli...|
|         2610|hamburger meat,do...|
|         2709|yogurt,other vege...|
|         2727|fish,frozen potat...|
|         2943|hamburger meat,wh...|
|         2974|whipped/sour crea...|
|         3681|specialty fat,oni...|
|         3797|fruit/vegetable j...|
|         3942|shopping bags,yog...|
|         3956|shopping bags,yog...|
|         4260|    soda,brown bread|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
items = dfMembers.select("Items").rdd.flatMap(list).map(lambda x: set(x.split(','))).reduce(lambda x, y: x.union(y))
items = list(items)
items.sort()

In [ ]:
items

['Instant food products',
 'UHT-milk',
 'abrasive cleaner',
 'artif. sweetener',
 'bags',
 'baking powder',
 'bathroom cleaner',
 'beef',
 'berries',
 'beverages',
 'bottled beer',
 'bottled water',
 'brandy',
 'brown bread',
 'butter',
 'butter milk',
 'cake bar',
 'candles',
 'candy',
 'canned beer',
 'canned fish',
 'canned fruit',
 'canned vegetables',
 'cat food',
 'cereals',
 'chewing gum',
 'chicken',
 'chocolate',
 'chocolate marshmallow',
 'citrus fruit',
 'cleaner',
 'cling film/bags',
 'cocoa drinks',
 'coffee',
 'condensed milk',
 'cooking chocolate',
 'cookware',
 'cream',
 'cream cheese ',
 'curd',
 'curd cheese',
 'decalcifier',
 'dental care',
 'dessert',
 'detergent',
 'dish cleaner',
 'dishes',
 'dog food',
 'domestic eggs',
 'female sanitary products',
 'finished products',
 'fish',
 'flour',
 'flower (seeds)',
 'flower soil/fertilizer',
 'frankfurter',
 'frozen chicken',
 'frozen dessert',
 'frozen fish',
 'frozen fruits',
 'frozen meals',
 'frozen potato products',

In [ ]:
dictItems = {k: v for v, k in enumerate(items)}

In [ ]:
dictItems

{'Instant food products': 0,
 'UHT-milk': 1,
 'abrasive cleaner': 2,
 'artif. sweetener': 3,
 'bags': 4,
 'baking powder': 5,
 'bathroom cleaner': 6,
 'beef': 7,
 'berries': 8,
 'beverages': 9,
 'bottled beer': 10,
 'bottled water': 11,
 'brandy': 12,
 'brown bread': 13,
 'butter': 14,
 'butter milk': 15,
 'cake bar': 16,
 'candles': 17,
 'candy': 18,
 'canned beer': 19,
 'canned fish': 20,
 'canned fruit': 21,
 'canned vegetables': 22,
 'cat food': 23,
 'cereals': 24,
 'chewing gum': 25,
 'chicken': 26,
 'chocolate': 27,
 'chocolate marshmallow': 28,
 'citrus fruit': 29,
 'cleaner': 30,
 'cling film/bags': 31,
 'cocoa drinks': 32,
 'coffee': 33,
 'condensed milk': 34,
 'cooking chocolate': 35,
 'cookware': 36,
 'cream': 37,
 'cream cheese ': 38,
 'curd': 39,
 'curd cheese': 40,
 'decalcifier': 41,
 'dental care': 42,
 'dessert': 43,
 'detergent': 44,
 'dish cleaner': 45,
 'dishes': 46,
 'dog food': 47,
 'domestic eggs': 48,
 'female sanitary products': 49,
 'finished products': 50,
 '

In [ ]:
from pyspark.ml.linalg import Vectors

def basket2vector(member, basket, dictItems):
  dictItems_1 = []
  indexItems = []
  basket_list = basket.split(",")
  basket_list.sort()
  for item in basket_list:
    indexItems.append(dictItems[item])
    dictItems_1.append(1.0)
  vector = Vectors.sparse(len(dictItems), indexItems, dictItems_1)
  return vector

In [ ]:
print(basket2vector(dfMembers.first()["Member_number"],
                    dfMembers.first()["Items"],
                    dictItems))

(164,[29,33],[1.0,1.0])


In [ ]:
dfMembers = dfMembers.rdd.map(lambda x: (x[0], basket2vector(x[0],x[1],dictItems))).toDF(["Member_number","Items"])

In [ ]:
dfMembers.show()

+-------------+--------------------+
|Member_number|               Items|
+-------------+--------------------+
|         1249|(164,[29,33],[1.0...|
|         1381|(164,[33,39,99,10...|
|         1440|(164,[27,63,99,15...|
|         1659|(164,[46,62,142,1...|
|         1789|(164,[17,67],[1.0...|
|         1922|(164,[11,27,40,99...|
|         2226|(164,[11,127,132,...|
|         2237|(164,[0,11,53,132...|
|         2351|(164,[30,130],[1....|
|         2542|(164,[11,132],[1....|
|         2610|(164,[10,48,67],[...|
|         2709|(164,[11,62,99,16...|
|         2727|(164,[10,51,61,63...|
|         2943|(164,[53,67,102,1...|
|         2974|(164,[8,11,62,102...|
|         3681|(164,[46,97,143,1...|
|         3797|(164,[63,129,156,...|
|         3942|(164,[0,20,34,45,...|
|         3956|(164,[20,27,67,86...|
|         4260|(164,[13,135],[1....|
+-------------+--------------------+
only showing top 20 rows



# Câu 5: Similar baskets

In [ ]:
from pyspark.ml.feature import MinHashLSH
mh = MinHashLSH(inputCol="Items", outputCol="Hashes")
mh.setNumHashTables(8)

MinHashLSH_cedeafb4fa65

In [ ]:
model = mh.fit(dfMembers)
dfMinHashLSH = model.transform(dfMembers)

In [ ]:
dfMinHashLSH.show()

+-------------+--------------------+--------------------+
|Member_number|               Items|              Hashes|
+-------------+--------------------+--------------------+
|         1249|(164,[29,33],[1.0...|[[1.01068404E8], ...|
|         1381|(164,[33,39,99,10...|[[1.7126837E7], [...|
|         1440|(164,[27,63,99,15...|[[1.66640022E8], ...|
|         1659|(164,[46,62,142,1...|[[4.14510634E8], ...|
|         1789|(164,[17,67],[1.0...|[[1.431804377E9],...|
|         1922|(164,[11,27,40,99...|[[3.8984043E7], [...|
|         2226|(164,[11,127,132,...|[[7.9211198E7], [...|
|         2237|(164,[0,11,53,132...|[[6.8282595E7], [...|
|         2351|(164,[30,130],[1....|[[2.83367398E8], ...|
|         2542|(164,[11,132],[1....|[[7.9211198E7], [...|
|         2610|(164,[10,48,67],[...|[[8.19335777E8], ...|
|         2709|(164,[11,62,99,16...|[[7.9211198E7], [...|
|         2727|(164,[10,51,61,63...|[[1.7126837E7], [...|
|         2943|(164,[53,67,102,1...|[[1.154635213E9],...|
|         2974

In [ ]:
dfFillter = model.approxSimilarityJoin(dfMembers,dfMembers,0.5, distCol="JaccardDistance")
dfFillter.filter("JaccardDistance > 0").show()

+--------------------+--------------------+-------------------+
|            datasetA|            datasetB|    JaccardDistance|
+--------------------+--------------------+-------------------+
|{3299, (164,[7,15...|{3509, (164,[7,11...|0.33333333333333337|
|{4963, (164,[10,2...|{2111, (164,[10,9...|                0.4|
|{2140, (164,[10,1...|{1405, (164,[10,9...|0.33333333333333337|
|{3860, (164,[48,1...|{3758, (164,[48,1...|0.33333333333333337|
|{4139, (164,[11,2...|{2835, (164,[11,4...|0.33333333333333337|
|{3649, (164,[27,5...|{1325, (164,[27,9...|                0.4|
|{1001, (164,[119,...|{3417, (164,[119,...|0.33333333333333337|
|{3928, (164,[29,4...|{1509, (164,[29,4...|0.33333333333333337|
|{4812, (164,[119,...|{3496, (164,[0,11...|0.33333333333333337|
|{3714, (164,[14,4...|{2911, (164,[14,4...|0.33333333333333337|
|{3215, (164,[11,1...|{3817, (164,[11,1...|0.33333333333333337|
|{3356, (164,[14,6...|{3836, (164,[14,8...|                0.4|
|{3376, (164,[38,9...|{2496, (164,[38,1.

In [ ]:
model.approxNearestNeighbors(dfMembers, dfMembers.first()['Items'], 5).show()

+-------------+--------------------+--------------------+-------------------+
|Member_number|               Items|              Hashes|            distCol|
+-------------+--------------------+--------------------+-------------------+
|         1249|(164,[29,33],[1.0...|[[1.01068404E8], ...|                0.0|
|         4174|(164,[29,33],[1.0...|[[1.01068404E8], ...|                0.0|
|         2199|(164,[29,33,119],...|[[1.01068404E8], ...|0.33333333333333337|
|         4771|(164,[29,33,88,14...|[[1.01068404E8], ...|                0.5|
|         2708|    (164,[33],[1.0])|[[1.01068404E8], ...|                0.5|
+-------------+--------------------+--------------------+-------------------+

